In [71]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor,  BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor


from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import altair as alt
import datetime as dt

In [72]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [73]:
def read_data(stock='TSLA', period = "7d", interval = "2m"):
    '''
    reading the data
    '''


    df = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,)


    return df

In [101]:
df1 = read_data(stock='TSLA').reset_index()

[*********************100%***********************]  1 of 1 completed


In [95]:
df2 = df1.copy()

In [96]:
# here is the tail of the df which has the most recent data
df1.tail()

,Datetime,Open,High,Low,Close,Adj Close,Volume
1360,2020-11-10 15:50:00-05:00,409.920013,410.850006,409.200012,409.269989,409.269989,140984
1361,2020-11-10 15:52:00-05:00,409.248291,409.760010,408.730011,409.570007,409.570007,109845
1362,2020-11-10 15:54:00-05:00,409.626312,410.119995,409.114288,409.730011,409.730011,107881
1363,2020-11-10 15:56:00-05:00,409.809998,410.820007,409.510010,410.109985,410.109985,171583
1364,2020-11-10 15:58:00-05:00,410.070007,410.880005,409.950012,410.410004,410.410004,231045


### Models

In [77]:
# We have the follwoing model already established

In [100]:
def test_stock(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    df = dfx.copy()
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [85]:
# now let's see the power of feature engineering:
#we will add the daily return as a feature for this model

In [105]:
def test_stock_w_feature(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    #print(len(df))
    df = dfx.copy()
    df['f01'] = df['Close']/df['Open']-1
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    




### Performance Comparision

In [102]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df1, intra = True)

1364   2020-11-10 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.995
error 0.7850662094742852
old price =  410.07000732421875
predicted =  410.76869542268764
difference =  0.6986880984688923


In [107]:
test_stock_w_feature(df1,intra =True)

1364   2020-11-10 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 1.000
error 0.02704167567744686
old price =  410.07000732421875
predicted =  410.4202450461427
difference =  0.35023772192397473


In [66]:
# Do we same the same gain if use a different modeel: Random Forest ?

In [109]:
def test_stock_grad(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """

    df = dfx.copy()

    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [110]:
def test_stock_grad_feature(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """


    df = dfx.copy()
    
    df['f01'] = df['Close']/df['Open']-1
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    


## Intra-day example

In [69]:
## Ridge model

In [111]:
# let's predict he clost price for TSLA after 5 minutes
test_stock_grad(df1,intra = True)

1364   2020-11-10 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.841
error 0.3374290437305777
old price =  410.07000732421875
predicted =  409.71956512451175
difference =  -0.35044219970700397


In [112]:
test_stock_grad_feature(df1, intra = True)

1364   2020-11-10 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.849
error 0.1722698507049234
old price =  410.07000732421875
predicted =  410.1792889404297
difference =  0.10928161621092158


> Again we noticed an improved performance when we include this feature

----

### Long term